# Sequential metasolver

The sequential metasolver implemented in discrete-optimization is a generic metasolver than is chaining other subsolvers, warmstarting them with the best solution found by the previous one.

It requires that the subsolvers (except the first one) are warmstartable which in discrete-optimization means deriving from `WarmstartMixin` and implementing the method `set_warm_start()`.

Thanks to the use of a list hyperparameter, we can make [Optuna](https://optuna.readthedocs.io/en/stable/) optimize the choice of subsolvers and their order.

To best understand what is a hyperparameter in discrete-optimization and how the library integrates with optuna, we recommend to first read the [tutorial dedicated to Optuna]().

As we will use a RCPSP problem to illustrate the use of the sequential metasolver, it could also be useful to take a look at the [RCPSP tutorial](../RCPSP%20tutorials/RCPSP-1%20Introduction.ipynb).

## Prerequisites

Concerning the python kernel to use for this notebook:
- If running locally, be sure to use an environment with discrete-optimization, minizinc, and optuna (and optionally optuna-dashboard);
- If running on colab, the next cell does it for you;
- If running on binder, the environment should be ready.


In [ ]:
# On Colab: install the library
on_colab = "google.colab" in str(get_ipython())
if on_colab:
    import os
    import sys  # noqa: avoid having this import removed by pycln

    !{sys.executable} -m pip install -U pip

    # uninstall google protobuf conflicting with ray and sb3
    ! pip uninstall -y protobuf

    # install dev version for dev doc, or release version for release doc
    !{sys.executable} -m pip install git+https://github.com/airbus/discrete-optimization@master#egg=discrete-optimization

    # install and configure minizinc
    !curl -o minizinc.AppImage -L https://github.com/MiniZinc/MiniZincIDE/releases/download/2.8.5/MiniZincIDE-2.8.5-x86_64.AppImage
    !chmod +x minizinc.AppImage
    !./minizinc.AppImage --appimage-extract
    os.environ["PATH"] = f"{os.getcwd()}/squashfs-root/usr/bin/:{os.environ['PATH']}"
    os.environ["LD_LIBRARY_PATH"] = (
        f"{os.getcwd()}/squashfs-root/usr/lib/:{os.environ['LD_LIBRARY_PATH']}"
    )

    # install optuna and optuna-dashboard
    !{sys.executable} -m pip install optuna optuna-dashboard

### Imports

In [ ]:
from __future__ import annotations

import logging
import os
import socket

import nest_asyncio
import numpy as np
from optuna.samplers import RandomSampler
from optuna.storages import JournalFileStorage, JournalStorage

from discrete_optimization.datasets import fetch_data_from_psplib
from discrete_optimization.generic_tools.callbacks.loggers import ObjectiveLogger
from discrete_optimization.generic_tools.cp_tools import ParametersCp
from discrete_optimization.generic_tools.hyperparameters.hyperparameter import SubBrick
from discrete_optimization.generic_tools.ls.local_search import (
    ModeMutation,
    RestartHandlerLimit,
)
from discrete_optimization.generic_tools.ls.simulated_annealing import (
    SimulatedAnnealing,
    TemperatureSchedulingFactor,
)
from discrete_optimization.generic_tools.mutations.mixed_mutation import (
    BasicPortfolioMutation,
)
from discrete_optimization.generic_tools.mutations.mutation_catalog import (
    get_available_mutations,
)
from discrete_optimization.generic_tools.optuna.utils import (
    generic_optuna_experiment_monoproblem,
)
from discrete_optimization.generic_tools.sequential_metasolver import (
    SequentialMetasolver,
)
from discrete_optimization.rcpsp.parser import get_data_available, parse_file
from discrete_optimization.rcpsp.solvers.cp_mzn import CpRcpspSolver
from discrete_optimization.rcpsp.solvers.cpsat import CpSatRcpspSolver
from discrete_optimization.rcpsp.solvers.lp import GurobiMultimodeRcpspSolver
from discrete_optimization.rcpsp.solvers.pile import PileRcpspSolver

# patch asyncio so that applications using async functions can run in jupyter
nest_asyncio.apply()

# set logging level
logging.basicConfig(level=logging.INFO, format="%(asctime)s:%(message)s")

### Download datasets

If not yet available, we import the datasets from [psplib](https://www.om-db.wi.tum.de/psplib/data.html).

In [ ]:
needed_datasets = ["j1201_1.sm"]
download_needed = False
try:
    files_available_paths = get_data_available()
    for dataset in needed_datasets:
        if len([f for f in files_available_paths if dataset in f]) == 0:
            download_needed = True
            break
except:
    download_needed = True

if download_needed:
    fetch_data_from_psplib()

In [ ]:
files_available = get_data_available()
file = [f for f in files_available if "j301_1.sm" in f][0]
problem = parse_file(file)

## Simple use

We first will chain
- a greedy solver (to initialize the process)
- a local search solver (to improve the solution locally)
- a cpsat ortools solver (starting from this improved solution)

For that we specify the list of bricks to use with classes and keyword arguments.

In [ ]:
# kwargs SA
solution = problem.get_dummy_solution()
_, list_mutation = get_available_mutations(problem, solution)
list_mutation = [
    mutate[0].build(problem, solution, **mutate[1]) for mutate in list_mutation
]
mixed_mutation = BasicPortfolioMutation(list_mutation, np.ones((len(list_mutation))))
restart_handler = RestartHandlerLimit(3000)
temperature_handler = TemperatureSchedulingFactor(1000, restart_handler, 0.99)

# kwargs cpsat
parameters_cp = ParametersCp.default_cpsat()

# list of bricks
list_subbricks = [
    SubBrick(cls=PileRcpspSolver, kwargs=dict()),
    SubBrick(
        cls=SimulatedAnnealing,
        kwargs=dict(
            mutator=mixed_mutation,
            restart_handler=restart_handler,
            temperature_handler=temperature_handler,
            mode_mutation=ModeMutation.MUTATE,
            nb_iteration_max=5000,
        ),
    ),
    SubBrick(
        cls=CpSatRcpspSolver,
        kwargs=dict(parameters_cp=parameters_cp, time_limit=20),
    ),
]

# solve
solver = SequentialMetasolver(problem=problem, list_subbricks=list_subbricks)
result_storage = solver.solve(
    callbacks=[
        ObjectiveLogger(
            step_verbosity_level=logging.INFO, end_verbosity_level=logging.INFO
        ),
    ],
)
solution, fit = result_storage.get_best_solution_fit()
print("makespan: ", fit)

## Using dynamic kwargs

The kwargs used for a subsolver could potentially be dynamically defined according to the best solution found by the previous one.
This is possible thanks to `SubBrick.kwargs_from_solution` which maps an argument name to a function taking a solution as argument and returning the desired value for the argument.

**Warning**: the next cell needs gurobi to be installed on your machine, with a suitable license.

**Note**: if you have only the free license coming with gurobipy installed from pypi with `pip install gurobipy`, the cell works if using the simpler (but perhaps less relevant) problem `j301.sm` with
```python
files_available = get_data_available()
file = [f for f in files_available if "j301_1.sm" in f][0]
problem = parse_file(file)
```

In [ ]:
# kwargs SA
solution = problem.get_dummy_solution()
_, list_mutation = get_available_mutations(problem, solution)
list_mutation = [
    mutate[0].build(problem, solution, **mutate[1]) for mutate in list_mutation
]
mixed_mutation = BasicPortfolioMutation(list_mutation, np.ones((len(list_mutation))))
restart_handler = RestartHandlerLimit(3000)
temperature_handler = TemperatureSchedulingFactor(1000, restart_handler, 0.99)

# list of bricks
list_subbricks = [
    SubBrick(cls=PileRcpspSolver, kwargs=dict()),
    SubBrick(
        cls=SimulatedAnnealing,
        kwargs=dict(
            mutator=mixed_mutation,
            restart_handler=restart_handler,
            temperature_handler=temperature_handler,
            mode_mutation=ModeMutation.MUTATE,
            nb_iteration_max=5000,
        ),
    ),
    SubBrick(
        cls=GurobiMultimodeRcpspSolver,
        kwargs=dict(),
        kwargs_from_solution=dict(
            start_solution=lambda sol: sol,
            max_horizon=lambda sol: int(1.2 * problem.evaluate(sol)["makespan"]),
        ),
    ),
]

# solve
solver = SequentialMetasolver(problem=problem, list_subbricks=list_subbricks)
result_storage = solver.solve(
    callbacks=[
        ObjectiveLogger(
            step_verbosity_level=logging.INFO, end_verbosity_level=logging.INFO
        ),
    ],
)
solution, fit = result_storage.get_best_solution_fit()

## Tuning with optuna

The sequential metasolver defines a list hyperparameter that can be used to choose the sequence of subsolvers to chain.
To simplify our work, we use the utility function `generic_optuna_experiment_monoproblem` that will
- create the optuna study,
- define the proper objective function for it,
- update the available choices for the subsolvers,
- run the study (ie optimize the objective function by tuning the available hyperparameters).

Note that as the first subsolver can be a non-warmstartable solver, the parameter `list_subbricks` has been split into 2 hyperparameters:
- `subsolver_0`: the first subsolver
- `next_subsolvers`: the other ones, for which `choices` must contain only warmstartable subsolvers.

### Optuna dashboard
Before running the study, we launch optuna-dashboard to watch the optimization progress with

    optuna-dashboard optuna-journal.log

The next cell do it according to your jupyter environment:
- if running locally, we need to install optuna-dashboard and run it (in a separate process);
- if running on colab, we make use of `google.colab.output` as suggested [here](https://stackoverflow.com/a/76033378);
- if running on binder, we sadly did not succed in using `jupyter-server-proxy` to access to the served dashboard, as done for tensorboard [here](https://github.com/binder-examples/tensorboard).


In [ ]:
on_colab = "google.colab" in str(get_ipython())  # running on colab?
on_binder = socket.gethostname().startswith(
    "jupyter-"
)  # running on binder? (not 100% sure but rather robust)


# ensure preexistence of optuna log journal
storage_path = "./optuna-journal.log"
storage = JournalStorage(JournalFileStorage(storage_path))


def start_optuna_dashboard(port=1234):
    import threading
    import time
    from wsgiref.simple_server import make_server

    from optuna_dashboard import wsgi

    app = wsgi(storage)
    httpd = make_server("localhost", port, app)
    thread = threading.Thread(target=httpd.serve_forever)
    thread.start()
    time.sleep(3)  # Wait until the server startup
    return port


if on_colab:
    port = start_optuna_dashboard()
    from google.colab import output

    print("Visit optuna-dashboard on:")
    output.serve_kernel_port_as_window(port, path="/dashboard/")

elif on_binder:
    print("Not yet working on binder...")
else:
    try:
        import optuna_dashboard  # nopycln: import
    except ImportError:
        !pip install optuna-dashboard
    port = start_optuna_dashboard()
    print(f"Visit optuna-dashboard on http://localhost:{port}/")

### Study with default sampler (TPESampler)

In [ ]:
# we want only to test sequential metasolver
solvers_to_test = [SequentialMetasolver]


# kwargs SA
solution = problem.get_dummy_solution()
_, list_mutation = get_available_mutations(problem, solution)
list_mutation = [
    mutate[0].build(problem, solution, **mutate[1]) for mutate in list_mutation
]
mixed_mutation = BasicPortfolioMutation(list_mutation, np.ones((len(list_mutation))))
restart_handler = RestartHandlerLimit(3000)
temperature_handler = TemperatureSchedulingFactor(1000, restart_handler, 0.99)

# fixed kwargs by subsolver type
fixed_hyperparameters_by_subbrick = {  # fixed kwargs by subsolver
    SimulatedAnnealing: dict(
        mutator=mixed_mutation,
        restart_handler=restart_handler,
        temperature_handler=temperature_handler,
        mode_mutation=ModeMutation.MUTATE,
        nb_iteration_max=5000,
    ),
    CpSatRcpspSolver: dict(parameters_cp=ParametersCp.default_cpsat(), time_limit=20),
    CpRcpspSolver: dict(parameters_cp=ParametersCp.default_cpsat(), time_limit=20),
}

# specify more options for sequential metasolver hyperparameter
suggest_optuna_kwargs_by_name_by_solver = {
    SequentialMetasolver: dict(
        subsolver_0=dict(
            choices=[
                PileRcpspSolver,
                CpSatRcpspSolver,
                CpRcpspSolver,
            ],  # possible subsolver class (even not warmstartable)
            fixed_hyperparameters_by_subbrick=fixed_hyperparameters_by_subbrick,  # fixed kwargs by subsolver
        ),
        next_subsolvers=dict(
            length_low=1,  # lower bound for list length
            length_high=2,  # upper bound for list length
            choices=[
                SimulatedAnnealing,
                CpSatRcpspSolver,
            ],  # possible subsolver class, warmstartable only
            fixed_hyperparameters_by_subbrick=fixed_hyperparameters_by_subbrick,  # fixed kwargs by subsolver
        ),
    )
}


study = generic_optuna_experiment_monoproblem(
    problem=problem,
    study_basename=f"sequential_metasolver-rcpsp-{os.path.basename(file)}",
    n_trials=100,
    solvers_to_test=solvers_to_test,
    suggest_optuna_kwargs_by_name_by_solver=suggest_optuna_kwargs_by_name_by_solver,
)

We notice that starting from the 13-th trial, the default optuna sampler got stuck into reusing the same set of hyperparameters even though not every possibility have been tried.

### Study with pure random sampler 
We try to use a purely random random sampler to avoid being stuck in a working sequence of subsolvers even before trying all options of subsolvers.

In [ ]:
# we want only to test sequential metasolver
solvers_to_test = [SequentialMetasolver]


# kwargs SA
solution = problem.get_dummy_solution()
_, list_mutation = get_available_mutations(problem, solution)
list_mutation = [
    mutate[0].build(problem, solution, **mutate[1]) for mutate in list_mutation
]
mixed_mutation = BasicPortfolioMutation(list_mutation, np.ones((len(list_mutation))))
restart_handler = RestartHandlerLimit(3000)
temperature_handler = TemperatureSchedulingFactor(1000, restart_handler, 0.99)

# fixed kwargs by subsolver type
fixed_hyperparameters_by_subbrick = {  # fixed kwargs by subsolver
    SimulatedAnnealing: dict(
        mutator=mixed_mutation,
        restart_handler=restart_handler,
        temperature_handler=temperature_handler,
        mode_mutation=ModeMutation.MUTATE,
        nb_iteration_max=5000,
    ),
    CpSatRcpspSolver: dict(parameters_cp=ParametersCp.default_cpsat(), time_limit=20),
    CpRcpspSolver: dict(parameters_cp=ParametersCp.default_cpsat(), time_limit=20),
}

# specify more options for sequential metasolver hyperparameter
suggest_optuna_kwargs_by_name_by_solver = {
    SequentialMetasolver: dict(
        subsolver_0=dict(
            choices=[
                PileRcpspSolver,
                CpSatRcpspSolver,
                CpRcpspSolver,
            ],  # possible subsolver class (even not warmstartable)
            fixed_hyperparameters_by_subbrick=fixed_hyperparameters_by_subbrick,  # fixed kwargs by subsolver
        ),
        next_subsolvers=dict(
            length_low=1,  # lower bound for list length
            length_high=2,  # upper bound for list length
            choices=[
                SimulatedAnnealing,
                CpSatRcpspSolver,
            ],  # possible subsolver class, warmstartable only
            fixed_hyperparameters_by_subbrick=fixed_hyperparameters_by_subbrick,  # fixed kwargs by subsolver
        ),
    )
}


study = generic_optuna_experiment_monoproblem(
    problem=problem,
    study_basename=f"sequential_metasolver-rcpsp-{os.path.basename(file)}",
    n_trials=100,
    sampler=RandomSampler(),
    solvers_to_test=solvers_to_test,
    suggest_optuna_kwargs_by_name_by_solver=suggest_optuna_kwargs_by_name_by_solver,
)